In [1]:
import geemap
import ee

Map = geemap.Map()
Map

Map(center=[20, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(Togg…

In [2]:
def maskS2clouds(image):
    qa = image.select('QA60')
    cloudBitMask = 1 << 10
    cirrusBitMask = 1 << 11
    mask = qa.bitwiseAnd(cloudBitMask).eq(0).And(qa.bitwiseAnd(cirrusBitMask).eq(0))
    return image.updateMask(mask).select("B.*").copyProperties(image, ["system:time_start"])

In [3]:
def addTimeImag(image):
    timeImage = image.metadata('system:time_start').rename('timestamp')
    timeImageMasked = timeImage.updateMask(image.mask().select(0))
    return image.addBands(timeImageMasked)

In [11]:
shp = 'D:/Data/parper/data/shp/JX.shp'
shp_ee = geemap.shp_to_ee(shp)
geometry = shp_ee.geometry()

Map.addLayer(geometry, {'color': 'red'}, 'Farm')
Map.centerObject(geometry)

In [12]:
s2 = ee.ImageCollection("COPERNICUS/S2_SR")\
        .filterDate('2019-01-01', '2020-01-01')\
        .filterBounds(geometry).map(maskS2clouds).map(addTimeImag)

In [17]:
def filter_date(i):
    i = ee.Number(i)
    img = s2.filter(ee.Filter.calendarRange(i, i.add(10), 'day_of_year'))
    return img.mosaic().copyProperties(img.first(), ["system:time_start"])

In [18]:
time_list = ee.List.sequence(0, 365, 10)

In [19]:
s2_year_col = ee.ImageCollection(time_list.map(filter_date))

In [22]:
def image_join(filtered):
    days = 60
    millis = ee.Number(days).multiply(1000*60*60*24)
    maxDiffFilter = ee.Filter.maxDifference(**{
      'difference': millis,
      'leftField': 'system:time_start',
      'rightField': 'system:time_start'
    })
    lessEqFilter = ee.Filter.lessThanOrEquals(**{
      'leftField': 'system:time_start',
      'rightField': 'system:time_start'
    })
    greaterEqFilter = ee.Filter.greaterThanOrEquals(**{
      'leftField': 'system:time_start',
      'rightField': 'system:time_start'
    })

    filter1 = ee.Filter.And(maxDiffFilter, lessEqFilter)

    join1 = ee.Join.saveAll(**{
      'matchesKey': 'after',
      'ordering': 'system:time_start',
      'ascending': False})

    join1Result = join1.apply(**{
      'primary': filtered,
      'secondary': filtered,
      'condition': filter1
    })
    filter2 = ee.Filter.And(maxDiffFilter, greaterEqFilter)

    join2 = ee.Join.saveAll(**{
      'matchesKey': 'before',
      'ordering': 'system:time_start',
      'ascending': True})

    join2Result = join2.apply(**{
      'primary': join1Result,
      'secondary': join1Result,
      'condition': filter2
    })
    return join2Result

In [23]:
s2_join = image_join(s2_year_col)

In [25]:
def interpolateImages(image):
    image = ee.Image(image)
    beforeImages = ee.List(image.get('before'))
    beforeMosaic = ee.ImageCollection.fromImages(beforeImages).mosaic()
    afterImages = ee.List(image.get('after'))
    afterMosaic = ee.ImageCollection.fromImages(afterImages).mosaic()
    
    t1 = beforeMosaic.select('timestamp').rename('t1')
    t2 = afterMosaic.select('timestamp').rename('t2')
    t = image.metadata('system:time_start').rename('t')
    timeImage = ee.Image.cat([t1, t2, t])
    timeRatio = timeImage.expression('(t - t1) / (t2 - t1)', {
                't': timeImage.select('t'),
                't1': timeImage.select('t1'),
                't2': timeImage.select('t2'),
                })
    interpolated = beforeMosaic.add((afterMosaic.subtract(beforeMosaic).multiply(timeRatio)))
    result = image.unmask(interpolated)
    return result.copyProperties(image, ['system:time_start'])


In [26]:
interpolatedCol = ee.ImageCollection(s2_join.map(interpolateImages))

In [27]:
Map.addLayer(interpolatedCol.first(), {'min':0, 'max':3000}, 'img')

EEException: User memory limit exceeded.